# Import bibliotek i danych

In [16]:
import pandas as pd
import numpy as np
import pickle
import statsmodels.api as sm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn import neighbors

df=pd.read_csv("data_fe.csv")
df=df.drop(columns=['Unnamed: 0'])
features=df.columns.tolist()
features.remove('stars')
target='stars'

# Oszacowanie modelu kNN

In [9]:
def CVTestKNN(nFolds = 5, randomState=2024, debug=False, *args, **kwargs):
    kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)
    # Listy z wynikami
    testResults = []
    trainResults = []
    predictions = []
    indices = []
    # Pętla walidująca model na kolejnych foldach
    for train, test in kf.split(df.index.values):
        # Przygotowanie estymatora
        clf = neighbors.KNeighborsRegressor(*args, **kwargs)
        if debug:
            print(clf)
        # Trenowanie modelu
        clf.fit(df.iloc[train][features], df.iloc[train][target])
        # Przygotowanie prognoz dla zbioru treningowego i testowego
        predsTrain = clf.predict(df.iloc[train][features])
        preds = clf.predict(df.iloc[test][features])
        # Zachowajmy informacje o predykcjach dla tego foldu
        predictions.append(preds.tolist().copy())
        # Razem z indeksami w oryginalnym data frame
        indices.append(df.iloc[test].index.tolist().copy())
        # Policzenie RMSE dla foldów
        trainScore = metrics.mean_squared_error(df.iloc[train][target], predsTrain)**0.5
        testScore = metrics.mean_squared_error(df.iloc[test][target], preds)**0.5
        # Zapisanie wyników dla foldów
        trainResults.append(trainScore)
        testResults.append(testScore)
        # Informowanie o każdym foldzie razem z wynikami treningowymi możemy opcjonalnie wyświetlać w trakcie
        if debug:
            print("Train RMSE:", trainScore,
                  "Valid RMSE:", testScore)
        
    return trainResults, testResults, predictions, indices

In [17]:
trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=50, n_jobs=-1, p=2, debug=False)
print(np.mean(testResults))

0.25651391317177474


# Tuning hiperparametrów kNN

1. Liczba najbliższych sąsiadów - k

In [11]:
for k in [1, 3, 5, 10, 15, 30, 50, 100, 150, 200]:
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=k, n_jobs=-1, p=2)
    print(k, np.mean(trainResults), np.mean(testResults))

1 0.0 0.30081044226868137
3 0.1801776704743168 0.2575041102974596
5 0.20297742763910342 0.2506271471840048
10 0.22334966818424307 0.24726283974535637
15 0.23184142042316824 0.24904974090543947
30 0.24353507502423585 0.2518807303043076
50 0.25153642732533055 0.25651391317177474
100 0.2609955583567132 0.26356092682709814
150 0.2651254238254027 0.2670667462538331
200 0.26872643441994504 0.2703872320874618


Wydaje się, że najlepsza jest liczba 10-ciu


2. Wybór miary dystansu

In [18]:
for k in [5, 10, 15]:
    trainResults, testResults, predictions, indices = CVTestKNN(n_neighbors=k, n_jobs=-1, p=1)
    print(k, np.mean(testResults))

5 0.2400728402173018
10 0.23714754587900191
15 0.2371800109899858


Lepszy wynik dla miary Manhattan